### XML to Lat/Long

In [69]:
# # !pip install rasterio
# # !pip install pyproj
# import rasterio
# import numpy as np
# from affine import Affine
# from pyproj import Proj, transform

# fname = ''

# # Read raster
# with rasterio.open(fname) as r:
#     T0 = r.transform  # upper-left pixel corner affine transform
#     p1 = Proj(r.crs)
#     A = r.read()  # pixel values

# # All rows and columns
# cols, rows = np.meshgrid(np.arange(A.shape[2]), np.arange(A.shape[1]))

# # Get affine transform for pixel centres
# T1 = T0 * Affine.translation(0.5, 0.5)
# # Function to convert pixel row/column index (from 0) to easting/northing at centre
# rc2en = lambda r, c: (c, r) * T1

# # All eastings and northings (there is probably a faster way to do this)
# eastings, northings = np.vectorize(rc2en, otypes=[np.float, np.float])(rows, cols)

# # Project all longitudes, latitudes
# p2 = Proj(proj='latlong',datum='WGS84')
# longs, lats = transform(p1, p2, eastings, northings)

# Plotting with Bokeh (Cumulative Impact Score)

In [70]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure, ColumnDataSource
from bokeh.tile_providers import get_provider, Vendors
from bokeh.palettes import PRGn, RdYlGn, OrRd
from bokeh.transform import linear_cmap,factor_cmap
from bokeh.layouts import row, column
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter
import numpy as np
import pandas as pd

In [71]:
df = pd.read_csv('./data/Test Data.csv')
df.head()
df.dtypes

Location                                         object
Section                                          object
Lat                                             float64
Long                                            float64
Average cumulative impact score                 float64
Artisanal fishing                               float64
Demersal destructive fishing                    float64
Demersal nondestructive high bycatch fishing    float64
Demersal nondestructive low bycatch fishing     float64
Direct human impact                             float64
Inorganic pollution                             float64
Invasive species                                float64
Light pollution                                 float64
Nutrient pollution                              float64
Oil rigs                                        float64
Ocean acidification                             float64
Ocean-based pollution                           float64
Organic pollution                               

### Lat/long to mercator coords

In [72]:
# Define function to switch from lat/long to mercator coordinates
def x_coord(x, y):
    
    lat = x
    lon = y
    
    r_major = 6378137.000
    x = r_major * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale
    return (x, y)

# Define coord as tuple (lat,long)
df['coordinates'] = list(zip(df['Lat'], df['Long']))


# Obtain list of mercator coordinates
mercators = [x_coord(x, y) for x, y in df['coordinates'] ]

In [73]:
# Create mercator column in our df
df['mercator'] = mercators

# Split that column out into two separate cols - mercator_x and mercator_y
df[['mercator_x', 'mercator_y']] = df['mercator'].apply(pd.Series)

# Examine our modified DataFrame
df.head()

,Location,Section,Lat,Long,Average cumulative impact score,Artisanal fishing,Demersal destructive fishing,Demersal nondestructive high bycatch fishing,Demersal nondestructive low bycatch fishing,Direct human impact,...,Pelagic high bycatch fishing,Pelagic low bycatch fishing,Sea level rise,Sea surface temperature,Shipping,UV,coordinates,mercator,mercator_x,mercator_y
0,Manning-Hawkesbury,East Central AUSTRALIAn Shelf,-32.574900,152.371000,4.2285,0.0032,0.0266,0.0117,0.0070,0.0100,...,0.0003,0.0018,0.0263,1.6999,0.2199,0.7516,"(-32.5749, 152.371)","(16961862.13166189, -3839014.0322949826)",1.696186e+07,-3.839014e+06
1,Tweed-Moreton,East Central AUSTRALIAn Shelf,-28.175995,153.541672,4.0777,0.0021,0.0219,0.0086,0.0130,0.0108,...,0.0005,0.0136,0.0973,1.6373,0.1909,0.6975,"(-28.175995, 153.541672)","(17092180.74258783, -3271180.9073583093)",1.709218e+07,-3.271181e+06
2,Houtman,West Central AUSTRALIAn Shelf,-28.731028,113.840142,3.9096,0.0002,0.0092,0.0162,0.0028,0.0034,...,0.0010,0.0007,0.1358,1.5227,0.1812,0.7233,"(-28.73102797, 113.8401424)","(12672626.683801753, -3341456.2615737696)",1.267263e+07,-3.341456e+06
3,South AUSTRALIAn Gulfs,Southwest AUSTRALIAn Shelf,-34.184443,137.030539,3.8528,0.0008,0.0321,0.0325,0.0058,0.0131,...,0.0006,0.0005,0.3087,1.7451,0.1162,0.6277,"(-34.18444326, 137.0305387)","(15254169.791211968, -4053595.219408067)",1.525417e+07,-4.053595e+06
4,Leeuwin,Southwest AUSTRALIAn Shelf,-34.359162,115.138239,3.8312,0.0002,0.0094,0.0098,0.0025,0.0024,...,0.0004,0.0008,0.0786,1.6568,0.1138,0.7598,"(-34.35916175, 115.1382394)","(12817130.180842029, -4077131.235689167)",1.281713e+07,-4.077131e+06


### Preparing the plot

In [74]:
# Select tile set to use
chosentile = get_provider(Vendors.STAMEN_TONER)

# Choose palette
palette = OrRd[5]

# Tell Bokeh to use df as the source of the data
source = ColumnDataSource(data=df)

# Define color mapper - which column will define the colour of the data points
color_mapper = linear_cmap(field_name = 'Average cumulative impact score', \
                           palette = palette, low = df['Average cumulative impact score'].min(), \
                           high = df['Average cumulative impact score'].max())

# Set tooltips - these appear when we hover over a data point in our map, very nifty and very useful
tooltips = [("Impact Score","@{Average cumulative impact score}"), ("Location","@Location")]

In [75]:
# Create figure
p = figure(title = 'Average cumulative impact score', \
           x_axis_type="mercator", y_axis_type="mercator", \
           x_axis_label = 'Longitude', y_axis_label = 'Latitude', tooltips = tooltips)

# Add map tile
p.add_tile(chosentile)

# Add points using mercator coordinates
p.circle(x = 'mercator_x', y = 'mercator_y', color = color_mapper, source=source, size=20, fill_alpha = 0.7)

#Defines color bar
color_bar = ColorBar(color_mapper=color_mapper['transform'], 
                     formatter = NumeralTickFormatter(format='0.0[0000]'), 
                     label_standoff = 13, width=8, location=(0,0))
# Set color_bar location
p.add_layout(color_bar, 'right')

In [76]:
# Display in notebook
output_notebook()

# # Save as HTML
# output_file('avocado.html', title='Avocado Prices by region in the United States')

# Show map
show(p)

Loading BokehJS ...

# Graphing Stressor Indicators

In [77]:
color_mapper = linear_cmap(field_name = 'Inorganic pollution', \
                           palette = palette, low = df['Inorganic pollution'].max(), \
                           high = df['Inorganic pollution'].min())

tooltips = [("Inorganic pollution","@{Inorganic pollution}"), ("Location","@Location")]

p = figure(title = 'Inorganic pollution', \
           x_axis_type="mercator", y_axis_type="mercator", \
           x_axis_label = 'Longitude', y_axis_label = 'Latitude', tooltips = tooltips)

p.add_tile(chosentile)

p.circle(x = 'mercator_x', y = 'mercator_y', color = color_mapper, source=source, size=20, fill_alpha = 0.7)

color_bar = ColorBar(color_mapper=color_mapper['transform'], 
                     formatter = NumeralTickFormatter(format='0.0[0000]'), 
                     label_standoff = 13, width=8, location=(0,0))

p.add_layout(color_bar, 'right')

output_notebook()
show(p)

Loading BokehJS ...

In [78]:
color_mapper = linear_cmap(field_name = 'Nutrient pollution', \
                           palette = palette, low = df['Nutrient pollution'].max(), \
                           high = df['Nutrient pollution'].min())

tooltips = [("Nutrient pollution","@{Nutrient pollution}"), ("Location","@Location")]

p = figure(title = 'Nutrient pollution', \
           x_axis_type="mercator", y_axis_type="mercator", \
           x_axis_label = 'Longitude', y_axis_label = 'Latitude', tooltips = tooltips)

p.add_tile(chosentile)

p.circle(x = 'mercator_x', y = 'mercator_y', color = color_mapper, source=source, size=20, fill_alpha = 0.7)

color_bar = ColorBar(color_mapper=color_mapper['transform'], 
                     formatter = NumeralTickFormatter(format='0.0[0000]'), 
                     label_standoff = 13, width=8, location=(0,0))

p.add_layout(color_bar, 'right')

output_notebook()
show(p)

Loading BokehJS ...

In [79]:
color_mapper = linear_cmap(field_name = 'Ocean-based pollution', \
                           palette = palette, low = df['Ocean-based pollution'].max(), \
                           high = df['Nutrient pollution'].min())

tooltips = [("Ocean-based pollution","@{Ocean-based pollution}"), ("Location","@Location")]

p = figure(title = 'Ocean-based pollution', \
           x_axis_type="mercator", y_axis_type="mercator", \
           x_axis_label = 'Longitude', y_axis_label = 'Latitude', tooltips = tooltips)

p.add_tile(chosentile)

p.circle(x = 'mercator_x', y = 'mercator_y', color = color_mapper, source=source, size=20, fill_alpha = 0.7)

color_bar = ColorBar(color_mapper=color_mapper['transform'], 
                     formatter = NumeralTickFormatter(format='0.0[0000]'), 
                     label_standoff = 13, width=8, location=(0,0))

p.add_layout(color_bar, 'right')

output_notebook()
show(p)

Loading BokehJS ...

# Graphing Anthro. Impacts

In [80]:
color_mapper = linear_cmap(field_name = 'Direct human impact', \
                           palette = palette, low = df['Direct human impact'].max(), \
                           high = df['Direct human impact'].min())

tooltips = [("Direct human impact","@{Direct human impact}"), ("Location","@Location")]

p = figure(title = 'Direct human impact', \
           x_axis_type="mercator", y_axis_type="mercator", \
           x_axis_label = 'Longitude', y_axis_label = 'Latitude', tooltips = tooltips)

p.add_tile(chosentile)

p.circle(x = 'mercator_x', y = 'mercator_y', color = color_mapper, source=source, size=20, fill_alpha = 0.7)

color_bar = ColorBar(color_mapper=color_mapper['transform'], 
                     formatter = NumeralTickFormatter(format='0.0[0000]'), 
                     label_standoff = 13, width=8, location=(0,0))

p.add_layout(color_bar, 'right')

output_notebook()
show(p)

Loading BokehJS ...

In [81]:
color_mapper = linear_cmap(field_name = 'Demersal destructive fishing', \
                           palette = palette, low = df['Demersal destructive fishing'].max(), \
                           high = df['Demersal destructive fishing'].min())

tooltips = [("Demersal destructive fishing","@{Demersal destructive fishing}"), ("Location","@Location")]

p = figure(title = 'Demersal destructive fishing', \
           x_axis_type="mercator", y_axis_type="mercator", \
           x_axis_label = 'Longitude', y_axis_label = 'Latitude', tooltips = tooltips)

p.add_tile(chosentile)

p.circle(x = 'mercator_x', y = 'mercator_y', color = color_mapper, source=source, size=20, fill_alpha = 0.7)

color_bar = ColorBar(color_mapper=color_mapper['transform'], 
                     formatter = NumeralTickFormatter(format='0.0[0000]'), 
                     label_standoff = 13, width=8, location=(0,0))

p.add_layout(color_bar, 'right')

output_notebook()
show(p)

Loading BokehJS ...